# DATA PROCESSING
by Pol Rifé & Gonzalo Córdova

In [1]:
# imports of all the necessary packages
import pandas as pd
import datetime
import re

Let's start by reading the whole Dataset with all the accidents recorded from 2010 to present:

In [2]:
df = pd.read_csv('./data/traffic-collision-data-from-2010-to-present.csv')
df.head(3)

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Premise Description,Address,Cross Street,Location,Zip Codes,Census Tracts,Precinct Boundaries,LA Specific Plans,Council Districts,Neighborhood Councils (Certified)
0,191323054,2019-11-30T00:00:00.000,2019-11-30T00:00:00.000,130,13,Newton,1385,997,TRAFFIC COLLISION,NaN,...,STREET,SAN PEDRO ST,SLAUSON AV,"{'longitude': '-118.2696', 'human_address': '{...",22352.0,786.0,858.0,7.0,13.0,46.0
1,192020666,2019-11-30T00:00:00.000,2019-11-30T00:00:00.000,15,20,Olympic,2054,997,TRAFFIC COLLISION,NaN,...,STREET,OLYMPIC BL,KINGSLEY AV,"{'longitude': '-118.3031', 'human_address': '{...",22723.0,617.0,646.0,NaN,12.0,89.0
2,191616992,2019-11-30T00:00:00.000,2019-11-30T00:00:00.000,230,16,Foothill,1669,997,TRAFFIC COLLISION,NaN,...,STREET,TUJUNGA CANYON BL,LA TUNA CANYON RD,"{'longitude': '-118.2704', 'human_address': '{...",3222.0,5.0,227.0,NaN,1.0,7.0


First of all, in order to obtain a single year from the original data set, we have to expand the date in columns 'Monthday', 'Month', 'Year' and 'WeekDay' for future purposes. We drop 'Date Reported' and 'Date Ocurred' as we won't use the first one, and the second is already expanded.

In order to choose the year we want to use, we can check the total sum of nulls in the dataframe for each year.

As we could observe, 2013 is the year with less nulls int the data set. 

So, we end up selecting **2013**. This choice has been made according to the low amount of nulls that we can observe and because of its weather, as this year offers some variety of weather forecast.

In [3]:
months = []
years = []
weekday = []
monthday = []
for i in range(len(df)):
    date = datetime.datetime.strptime(df.iloc[i,2], "%Y-%m-%dT%H:%M:%S.%f")
    months.append(date.month) 
    years.append(date.year)
    weekday.append(date.weekday())
    monthday.append(date.day)
df['Month'] = months
df['Year'] = years
df['WeekDay'] = weekday
df['Monthday'] = monthday

df.drop('Date Occurred', axis = 1, inplace = True)
df.drop('Date Reported', axis = 1, inplace = True)
df.head(5)

,DR Number,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,...,Zip Codes,Census Tracts,Precinct Boundaries,LA Specific Plans,Council Districts,Neighborhood Councils (Certified),Month,Year,WeekDay,Monthday
0,191323054,130,13,Newton,1385,997,TRAFFIC COLLISION,NaN,NaN,F,...,22352.0,786.0,858.0,7.0,13.0,46.0,11,2019,5,30
1,192020666,15,20,Olympic,2054,997,TRAFFIC COLLISION,NaN,40.0,M,...,22723.0,617.0,646.0,NaN,12.0,89.0,11,2019,5,30
2,191616992,230,16,Foothill,1669,997,TRAFFIC COLLISION,NaN,18.0,M,...,3222.0,5.0,227.0,NaN,1.0,7.0,11,2019,5,30
3,191824082,730,18,Southeast,1802,997,TRAFFIC COLLISION,0605,23.0,M,...,22352.0,800.0,358.0,7.0,14.0,45.0,11,2019,5,30
4,191616980,720,16,Foothill,1689,997,TRAFFIC COLLISION,NaN,NaN,M,...,19335.0,166.0,272.0,NaN,3.0,21.0,11,2019,5,30


In [4]:
df2013 = df.query('Year == 2013')
df2013.head(3)

,DR Number,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,...,Zip Codes,Census Tracts,Precinct Boundaries,LA Specific Plans,Council Districts,Neighborhood Councils (Certified),Month,Year,WeekDay,Monthday
322060,130224399,1405,2,Rampart,269,997,TRAFFIC COLLISION,3101 3401 3701 3004 3029 3026,32.0,M,...,23078.0,568.0,1378.0,54.0,11.0,68.0,12,2013,1,31
322061,131925431,1245,19,Mission,1984,997,TRAFFIC COLLISION,3004 3037 3030 3101 3401 3701 4026,24.0,F,...,19730.0,149.0,419.0,NaN,3.0,59.0,12,2013,1,31
322062,132120318,900,21,Topanga,2187,997,TRAFFIC COLLISION,3004 3037 3101 3701 4026,39.0,M,...,19346.0,311.0,1490.0,6.0,4.0,49.0,12,2013,1,31


Let's see the columns our single year DataFrame has:

In [5]:
df2013.keys()

Index(['DR Number', 'Time Occurred', 'Area ID', 'Area Name',
       'Reporting District', 'Crime Code', 'Crime Code Description',
       'MO Codes', 'Victim Age', 'Victim Sex', 'Victim Descent',
       'Premise Code', 'Premise Description', 'Address', 'Cross Street',
       'Location', 'Zip Codes', 'Census Tracts', 'Precinct Boundaries',
       'LA Specific Plans', 'Council Districts',
       'Neighborhood Councils (Certified)', 'Month', 'Year', 'WeekDay',
       'Monthday'],
      dtype='object')

In [6]:
df2013 = df2013.set_axis(['DR Number', 'Time_Occurred',
       'Area_ID', 'Area_Name', 'Reporting_District', 'Crime Code',
       'Crime Code Description', 'MO Codes', 'Victim_Age', 'Victim_Sex',
       'Victim_Descent', 'Premise Code', 'Premise Description', 'Address',
       'Cross Street', 'Location', 'Zip Codes', 'Census Tracts',
       'Precinct Boundaries', 'LA Specific Plans', 'Council Districts',
       'Neighborhood Councils (Certified)', 'Month', 'Year', 'WeekDay', 'Monthday'], axis=1, inplace=False)

df2013.head(3)

,DR Number,Time_Occurred,Area_ID,Area_Name,Reporting_District,Crime Code,Crime Code Description,MO Codes,Victim_Age,Victim_Sex,...,Zip Codes,Census Tracts,Precinct Boundaries,LA Specific Plans,Council Districts,Neighborhood Councils (Certified),Month,Year,WeekDay,Monthday
322060,130224399,1405,2,Rampart,269,997,TRAFFIC COLLISION,3101 3401 3701 3004 3029 3026,32.0,M,...,23078.0,568.0,1378.0,54.0,11.0,68.0,12,2013,1,31
322061,131925431,1245,19,Mission,1984,997,TRAFFIC COLLISION,3004 3037 3030 3101 3401 3701 4026,24.0,F,...,19730.0,149.0,419.0,NaN,3.0,59.0,12,2013,1,31
322062,132120318,900,21,Topanga,2187,997,TRAFFIC COLLISION,3004 3037 3101 3701 4026,39.0,M,...,19346.0,311.0,1490.0,6.0,4.0,49.0,12,2013,1,31


To continue pre-processing our dataset, we would like to change the format of the 'Time Occurred' column as it is not very clear to interpret. To do so, we will expand our dataset into two new columns which will contain the hour and minutes respectively. Then, 'Time Occurred' will also be dropped as it has already been expanded.

In [7]:
hours = []
minutes = []

for i in range (len(df2013)):
    number_str = str(df2013.iloc[i]['Time_Occurred'])
    zero_filled_number = number_str.zfill(4) #we add zeros in order to have strings of the same length
    time = pd.to_datetime(zero_filled_number, format='%H%M')
    hours.append(time.hour)
    minutes.append(time.minute)

df2013['Time_Occurred_h'] = hours
df2013['Time_Occurred_min'] = minutes
df2013.drop('Time_Occurred', axis = 1, inplace = True)
df2013.head(3)

,DR Number,Area_ID,Area_Name,Reporting_District,Crime Code,Crime Code Description,MO Codes,Victim_Age,Victim_Sex,Victim_Descent,...,Precinct Boundaries,LA Specific Plans,Council Districts,Neighborhood Councils (Certified),Month,Year,WeekDay,Monthday,Time_Occurred_h,Time_Occurred_min
322060,130224399,2,Rampart,269,997,TRAFFIC COLLISION,3101 3401 3701 3004 3029 3026,32.0,M,A,...,1378.0,54.0,11.0,68.0,12,2013,1,31,14,5
322061,131925431,19,Mission,1984,997,TRAFFIC COLLISION,3004 3037 3030 3101 3401 3701 4026,24.0,F,H,...,419.0,NaN,3.0,59.0,12,2013,1,31,12,45
322062,132120318,21,Topanga,2187,997,TRAFFIC COLLISION,3004 3037 3101 3701 4026,39.0,M,W,...,1490.0,6.0,4.0,49.0,12,2013,1,31,9,0


Before saving the dataframe in a csv file, we are going to drop useless columns and create useful ones. For example, we will create two more columns that stores de Latitud and the Longitud of the Location of each accident:

In [8]:
longitud = []
latitud = []
for i in range (0,len(df2013)):
    numbers = [float(s) for s in re.findall(r'-?\d+\.?\d*', df2013.iloc[i]['Location'])]
    longitud.append(numbers[0])
    latitud.append(numbers[1])
df2013['longitud'] = longitud
df2013['latitud'] = latitud
df2013.drop(columns=['Location','Premise Code','Address','Cross Street', 'MO Codes','Premise Description', 'LA Specific Plans','Precinct Boundaries', 'Crime Code Description','Census Tracts', 'Zip Codes','Crime Code', 'Reporting_District', 'Neighborhood Councils (Certified)', 'Council Districts'], inplace = True)

In [9]:
df2013.head(5)

,DR Number,Area_ID,Area_Name,Victim_Age,Victim_Sex,Victim_Descent,Month,Year,WeekDay,Monthday,Time_Occurred_h,Time_Occurred_min,longitud,latitud
322060,130224399,2,Rampart,32.0,M,A,12,2013,1,31,14,5,-118.2650,34.0496
322061,131925431,19,Mission,24.0,F,H,12,2013,1,31,12,45,-118.4553,34.2282
322062,132120318,21,Topanga,39.0,M,W,12,2013,1,31,9,0,-118.5881,34.1677
322063,141704025,17,Devonshire,61.0,M,O,12,2013,1,31,18,15,-118.4959,34.2703
322064,140204010,2,Rampart,44.0,F,O,12,2013,1,31,20,15,-118.2728,34.0526


Once we have our DataFrame cleaned, we will add some extra columns in order to have a more accurate information about each row of it. For instance, we would like to know if one record belongs to Weekend or not. To do so, we will add a boolean variable that states wether that day belongs to weekend or not. 
Moreover, it also might be interesting to have the real week names in our rows as Weekday can be sometimes misinterpretable. 

In [10]:
def weekend(n):
    return(int(n) in [5,6])


df2013["Weekend"] = df2013["WeekDay"].apply(weekend)

In [11]:
def days_names(n):
    if int(n) == 0: return "Monday"
    elif int(n) == 1: return "Tuesday"
    elif int(n) == 2: return "Wednesday"
    elif int(n) == 3: return "Thursday"
    elif int(n) == 4: return "Friday"
    elif int(n) == 5: return "Saturday"
    else: return "Sunday"
    
df2013["Week_names"] = df2013["WeekDay"].apply(days_names)
df2013.drop('Year', axis = 1, inplace = True)

Also, as we have done with days of the week, we will also store in our dataset the real names of each month:

In [15]:
def month_names(n):
    if int(n) == 1: return "January"
    elif int(n) == 2: return "February"
    elif int(n) == 3: return "March"
    elif int(n) == 4: return "April"
    elif int(n) == 5: return "May"
    elif int(n) == 6: return "June"
    elif int(n) == 7: return "July"
    elif int(n) == 8: return "August"
    elif int(n) == 9: return "September"
    elif int(n) == 10: return "October"
    elif int(n) == 11: return "November"
    else: return "December"
    
df2013["Month_names"] = df2013["Month"].apply(month_names)

In [16]:
df2013["Month_names"].unique()

array(['December', 'November', 'October', 'September', 'August', 'July',
       'June', 'May', 'April', 'March', 'February', 'January'],
      dtype=object)

Moreover, we will create an extra column which will store the whole date of eahc accidents as this information will also be useful for our further visualizations.

In [20]:
dayofyear = []
for i in range(0, len(df2013)):
    dayofyear.append('2013-' + str(df2013.iloc[i]["Month"]) + '-' + str(df2013.iloc[i]["Monthday"]))

df2013["DayOfYear"] = dayofyear
df2013['DayOfYear'] = pd.to_datetime(df2013['DayOfYear'])

Finally, we have observed that we had some accidents that were outliers as their coordinates were pointing outside of Los Angeles. In order to deal with this issue, we will remove these registers applying the following methodology: we will only keep those acccidents that have a longitud value higher than -118.7 and lower than -118.15 (since these longitudes are those corresponding to the margins of Los Angeles) and, in the same way, we will also keep just those records that have a latitude value higher than 33.5 but lower than 34.35.

In addition, we will rename the Area_Name column in order to have the same names in our dataset as those in the geojson file, as for other purposes, we would like to merge the information from both files.

In [24]:
df2013 = df2013[(df2013['longitud']<-118.15)&(df2013['longitud']>-118.7)]
df2013 = df2013[(df2013['latitud']<34.35)&(df2013['latitud']>33.5)]
df2013['Area_Name'] = df2013['Area_Name'].replace(['Pacific','Devonshire','West LA', 'Northeast', 'Hollenbeck', 'Mission', 'Foothill', 'N Hollywood', 'Southwest',
                                                            'Wilshire', 'Olympic', 'Topanga', 'Central', 'Harbor', 'West Valley', 'Newton', 'Van Nuys', 'Rampart', '77th Street',
                                                           'Hollywood', 'Southeast'],
                                                          ['PACIFIC', 'DEVONSHIRE', 'WEST LOS ANGELES', 'NORTHEAST', 'HOLLENBECK', 'MISSION', 'FOOTHILL', 'NORTH HOLLYWOOD',
                                                           'SOUTHWEST', 'WILSHIRE', 'OLYMPIC', 'TOPANGA', 'CENTRAL', 'HARBOR', 'WEST VALLEY', 'NEWTON', 'VAN NUYS', 'RAMPART', 
                                                           '77TH STREET', 'HOLLYWOOD', 'SOUTHEAST'])

Let's see the Data Set we have obtained:

In [25]:
df2013.head()

,DR Number,Area_ID,Area_Name,Victim_Age,Victim_Sex,Victim_Descent,Month,WeekDay,Monthday,Time_Occurred_h,Time_Occurred_min,longitud,latitud,Weekend,Week_names,Month_names,DayOfYear
322060,130224399,2,RAMPART,32.0,M,A,12,1,31,14,5,-118.2650,34.0496,False,Tuesday,December,2013-12-31
322061,131925431,19,MISSION,24.0,F,H,12,1,31,12,45,-118.4553,34.2282,False,Tuesday,December,2013-12-31
322062,132120318,21,TOPANGA,39.0,M,W,12,1,31,9,0,-118.5881,34.1677,False,Tuesday,December,2013-12-31
322063,141704025,17,DEVONSHIRE,61.0,M,O,12,1,31,18,15,-118.4959,34.2703,False,Tuesday,December,2013-12-31
322064,140204010,2,RAMPART,44.0,F,O,12,1,31,20,15,-118.2728,34.0526,False,Tuesday,December,2013-12-31


Once we have apply all the pre-processing tools in our dataset, we are ready to save it in a csv format and start with the visualitzations:

In [26]:
df2013.to_csv('./data/traffic_collisions2013',index=False)